In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import fastf1
import fastf1.logger
import numpy as np #nie masz czasu bawic sie w optymalizje
import optuna
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from telemetry_data_preprocessing import telemetry_computations

c:\Users\pat79\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
fastf1.logger.set_log_level('ERROR')
fastf1.Cache.enable_cache('cache')

In [ ]:
import fastf1
import pandas as pd

from telemetry_data_preprocessing import load_data

telemetry, X, y, baseline_df = load_data()

telemetry.head(5)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,DriverNumber,LapNumber
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,,0.155556,0 days 00:00:00,10076.374938,0.000000,1,16.0,False,1,interpolation,0.000194,3.748547e-08,OnTrack,-279.999631,3558.983052,-156.999885,1,1.0
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,,0.155556,0 days 00:00:00.110000,10130.000000,0.000000,1,16.0,False,1,car,0.000000,0.000000e+00,OnTrack,-279.999617,3558.982418,-156.999881,1,1.0
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,,0.155556,0 days 00:00:00.185000,9993.499884,0.000000,1,16.0,False,1,pos,-0.000268,-5.186355e-08,OnTrack,-280.000000,3559.000000,-157.000000,1,1.0
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,,0.155556,0 days 00:00:00.310000,9766.000000,0.000000,1,16.0,False,1,car,0.000000,0.000000e+00,OnTrack,-280.001327,3559.060983,-157.000413,1,1.0
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,,0.155556,0 days 00:00:00.445000,8956.001152,0.964284,1,16.0,False,1,pos,0.003325,6.439951e-07,OnTrack,-280.000000,3559.000000,-157.000000,1,1.0


In [6]:
baseline_df

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,LapTime_sec
0,0 days 01:04:15.902000,VER,1,0 days 00:01:39.019000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:42.414000,...,Red Bull Racing,0 days 01:02:36.652000,2023-03-05 15:03:38.501,12,1.0,False,,False,False,99.019
1,0 days 01:05:53.876000,VER,1,0 days 00:01:37.974000,2.0,1.0,NaT,NaT,0 days 00:00:31.342000,0 days 00:00:42.504000,...,Red Bull Racing,0 days 01:04:15.902000,2023-03-05 15:05:17.751,12,1.0,False,,False,True,97.974
3,0 days 01:09:09.858000,VER,1,0 days 00:01:37.976000,4.0,1.0,NaT,NaT,0 days 00:00:31.271000,0 days 00:00:42.642000,...,Red Bull Racing,0 days 01:07:31.882000,2023-03-05 15:08:33.731,1,1.0,False,,False,True,97.976
5,0 days 01:12:25.879000,VER,1,0 days 00:01:37.986000,6.0,1.0,NaT,NaT,0 days 00:00:31.341000,0 days 00:00:42.632000,...,Red Bull Racing,0 days 01:10:47.893000,2023-03-05 15:11:49.742,1,1.0,False,,False,True,97.986
6,0 days 01:14:03.900000,VER,1,0 days 00:01:38.021000,7.0,1.0,NaT,NaT,0 days 00:00:31.290000,0 days 00:00:42.763000,...,Red Bull Racing,0 days 01:12:25.879000,2023-03-05 15:13:27.728,1,1.0,False,,False,True,98.021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0 days 01:16:11.771000,PIA,81,0 days 00:01:41.295000,8.0,1.0,NaT,NaT,0 days 00:00:31.665000,0 days 00:00:44.590000,...,McLaren,0 days 01:14:30.476000,2023-03-05 15:15:32.325,1,16.0,False,,False,True,101.295
1051,0 days 01:17:53.305000,PIA,81,0 days 00:01:41.534000,9.0,1.0,NaT,NaT,0 days 00:00:31.989000,0 days 00:00:44.548000,...,McLaren,0 days 01:16:11.771000,2023-03-05 15:17:13.620,1,16.0,False,,False,True,101.534
1052,0 days 01:19:34.889000,PIA,81,0 days 00:01:41.584000,10.0,1.0,NaT,NaT,0 days 00:00:31.779000,0 days 00:00:44.617000,...,McLaren,0 days 01:17:53.305000,2023-03-05 15:18:55.154,1,16.0,False,,False,True,101.584
1053,0 days 01:21:16.241000,PIA,81,0 days 00:01:41.352000,11.0,1.0,NaT,NaT,0 days 00:00:31.894000,0 days 00:00:44.557000,...,McLaren,0 days 01:19:34.889000,2023-03-05 15:20:36.738,1,13.0,False,,False,True,101.352


In [7]:
telemetry.shape

(575797, 20)

In [8]:
X.head(5)

,LapNumber,SpeedI1,SpeedI2,SpeedFL
0,1.0,232.0,231.0,278.0
1,2.0,227.0,238.0,278.0
3,4.0,228.0,238.0,278.0
5,6.0,229.0,239.0,279.0
6,7.0,230.0,239.0,279.0


In [9]:
y.head(5)

0    99.019
1    97.974
3    97.976
5    97.986
6    98.021
Name: LapTime_sec, dtype: float64

In [10]:
telemetry = telemetry.drop(["DriverAhead", "RPM", "Source", "Status", "RelativeDistance"], axis = 1)

telemetry.DRS = telemetry.DRS.apply(lambda x: 1 if x in [10, 12, 14] else 0)

In [11]:
telemetry.head(5)

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,DriverNumber,LapNumber
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1,1.0
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1,1.0
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1,1.0
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1,1.0
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1,1.0


In [12]:
# Compute mean lap speed per (DriverNumber, LapNumber)
telemetry["mean_lap_speed"] = telemetry.groupby(["DriverNumber", "LapNumber"])["Speed"].transform("mean")

In [13]:
telemetry

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,DriverNumber,LapNumber,mean_lap_speed
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1,1.0,185.815557
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1,1.0,185.815557
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1,1.0,185.815557
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1,1.0,185.815557
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1,1.0,185.815557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575792,2023-03-05 15:23:59.014,0 days 01:22:57.165000,14.524167,0 days 00:01:40.924000,277.000000,7,99.0,False,0,5350.113611,-390.939673,1050.884326,-158.999997,81,12.0,190.927743
575793,2023-03-05 15:23:59.106,0 days 01:22:57.257000,9.438625,0 days 00:01:41.016000,278.149997,7,99.0,False,0,5357.235508,-388.000000,1111.000000,-159.000000,81,12.0,190.927743
575794,2023-03-05 15:23:59.174,0 days 01:22:57.325000,5.679722,0 days 00:01:41.084000,279.000000,7,99.0,False,0,5362.513611,-385.457412,1163.537539,-159.000000,81,12.0,190.927743
575795,2023-03-05 15:23:59.186,0 days 01:22:57.337000,5.679722,0 days 00:01:41.096000,279.119997,7,99.0,False,0,5363.446582,-385.000000,1173.000000,-159.000000,81,12.0,190.927743


In [ ]:
computations = telemetry_computations()

all_lon, all_lat = [], []
for (driver, lap), group in telemetry.groupby(['DriverNumber', 'LapNumber']):
    group = group.sort_values('Time')  # sort before computing derivatives
    lon_, lat_ = computations.compute_accelerations(telemetry=group)
    all_lon.append(lon_)
    all_lat.append(lat_)

all_lon_series = [pd.Series(arr) for arr in all_lon]
all_lat_series = [pd.Series(arr) for arr in all_lat]

telemetry['lon_acc'] = pd.concat(all_lon_series, ignore_index=True)
telemetry['lat_acc'] = pd.concat(all_lat_series, ignore_index=True)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.S

In [15]:
telemetry

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,DriverNumber,LapNumber,mean_lap_speed,lon_acc,lat_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1,1.0,185.815557,0.00000,0.00000
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1,1.0,185.815557,0.00000,0.00000
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1,1.0,185.815557,0.00000,-0.00000
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1,1.0,185.815557,0.10502,-0.00000
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1,1.0,185.815557,0.20226,-0.45947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575792,2023-03-05 15:23:59.014,0 days 01:22:57.165000,14.524167,0 days 00:01:40.924000,277.000000,7,99.0,False,0,5350.113611,-390.939673,1050.884326,-158.999997,81,12.0,190.927743,0.24534,0.04859
575793,2023-03-05 15:23:59.106,0 days 01:22:57.257000,9.438625,0 days 00:01:41.016000,278.149997,7,99.0,False,0,5357.235508,-388.000000,1111.000000,-159.000000,81,12.0,190.927743,0.35395,0.09953
575794,2023-03-05 15:23:59.174,0 days 01:22:57.325000,5.679722,0 days 00:01:41.084000,279.000000,7,99.0,False,0,5362.513611,-385.457412,1163.537539,-159.000000,81,12.0,190.927743,0.34333,0.03093
575795,2023-03-05 15:23:59.186,0 days 01:22:57.337000,5.679722,0 days 00:01:41.096000,279.119997,7,99.0,False,0,5363.446582,-385.000000,1173.000000,-159.000000,81,12.0,190.927743,0.28316,0.00376


In [16]:
telemetry['TimeNumberLapTime'] = telemetry.groupby(['DriverNumber', 'LapNumber']).cumcount() + 1
telemetry['TimeNumberLapCounts'] = telemetry.groupby(['DriverNumber', 'LapNumber'])['LapNumber'].transform('count')

telemetry['LapProgress'] = telemetry['TimeNumberLapTime'] / telemetry['TimeNumberLapCounts']

In [21]:
telemetry['DriverNumber'] = telemetry['DriverNumber'].astype(int)

In [22]:
telemetry[(telemetry['DriverNumber'] == 1.0) & (telemetry['LapNumber'] == 2)]

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,...,Y,Z,DriverNumber,LapNumber,mean_lap_speed,lon_acc,lat_acc,TimeNumberLapTime,TimeNumberLapCounts,LapProgress
762,2023-03-05 15:05:17.751,0 days 01:04:15.902000,NaN,0 days 00:00:00,280.142857,7,100.0,False,0,0.005385,...,1216.526172,-159.000000,1,2.0,196.531157,0.20226,-0.12800,1,738,0.001355
763,2023-03-05 15:05:17.846,0 days 01:04:15.997000,NaN,0 days 00:00:00.095000,280.821429,7,100.0,False,0,7.431896,...,1291.000000,-159.000000,1,2.0,196.531157,0.20226,-0.06583,2,738,0.002710
764,2023-03-05 15:05:18.011,0 days 01:04:16.162000,NaN,0 days 00:00:00.260000,282.000000,7,100.0,False,0,20.377778,...,1420.116930,-159.000000,1,2.0,196.531157,0.20226,0.10465,3,738,0.004065
765,2023-03-05 15:05:18.045,0 days 01:04:16.196000,NaN,0 days 00:00:00.294000,282.242856,7,100.0,False,0,23.052853,...,1447.000000,-159.000000,1,2.0,196.531157,0.20226,0.15240,4,738,0.005420
766,2023-03-05 15:05:18.205,0 days 01:04:16.356000,NaN,0 days 00:00:00.454000,283.385713,7,100.0,False,0,35.673121,...,1573.000000,-159.000000,1,2.0,196.531157,0.20226,0.05828,5,738,0.006775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2023-03-05 15:06:55.292,0 days 01:05:53.443000,NaN,0 days 00:01:37.541000,277.000000,7,100.0,False,0,5329.816111,...,868.936589,-158.999994,1,2.0,196.531157,0.18685,-0.06280,734,738,0.994580
1496,2023-03-05 15:06:55.386,0 days 01:05:53.537000,NaN,0 days 00:01:37.635000,277.939999,7,100.0,False,0,5337.086955,...,949.000000,-159.000000,1,2.0,196.531157,0.28316,-0.07556,735,738,0.995935
1497,2023-03-05 15:06:55.492,0 days 01:05:53.643000,NaN,0 days 00:01:37.741000,279.000000,7,100.0,False,0,5345.316111,...,1037.026452,-159.000001,1,2.0,196.531157,0.26723,0.15332,736,738,0.997290
1498,2023-03-05 15:06:55.546,0 days 01:05:53.697000,NaN,0 days 00:01:37.795000,279.449999,7,100.0,False,0,5349.518852,...,1081.000000,-159.000000,1,2.0,196.531157,0.23596,0.13963,737,738,0.998645


In [23]:
def divide_column_by_sign(df : pd.DataFrame, column : str) -> pd.DataFrame:
    
    df[f"positive_{column}"] = df[column].apply(lambda x: 0 if x < 0 else x)
    df[f"negative_{column}"] = df[column].apply(lambda x: 0 if x > 0 else x)
    
    return df

Obliczanie submodelu telemetrii w konteście całego okrążenia

In [24]:
lap_df = telemetry.copy()

# Compute absolute values
lap_df['abs_lat_acc'] = lap_df['lat_acc'].abs()
lap_df['abs_lon_acc'] = lap_df['lon_acc'].abs()

# Group by both driver and lap
lap_df['sum_lat_acc'] = lap_df.groupby(['DriverNumber', 'LapNumber'])['abs_lat_acc'].transform('sum')
lap_df['sum_lon_acc'] = lap_df.groupby(['DriverNumber', 'LapNumber'])['abs_lon_acc'].transform('sum')

In [25]:
lap_df

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,...,mean_lap_speed,lon_acc,lat_acc,TimeNumberLapTime,TimeNumberLapCounts,LapProgress,abs_lat_acc,abs_lon_acc,sum_lat_acc,sum_lon_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,...,185.815557,0.00000,0.00000,1,762,0.001312,0.00000,0.00000,840.76762,561.07970
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,...,185.815557,0.00000,0.00000,2,762,0.002625,0.00000,0.00000,840.76762,561.07970
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,...,185.815557,0.00000,-0.00000,3,762,0.003937,0.00000,0.00000,840.76762,561.07970
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,...,185.815557,0.10502,-0.00000,4,762,0.005249,0.00000,0.10502,840.76762,561.07970
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,...,185.815557,0.20226,-0.45947,5,762,0.006562,0.45947,0.20226,840.76762,561.07970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575792,2023-03-05 15:23:59.014,0 days 01:22:57.165000,14.524167,0 days 00:01:40.924000,277.000000,7,99.0,False,0,5350.113611,...,190.927743,0.24534,0.04859,755,759,0.994730,0.04859,0.24534,783.92720,524.94456
575793,2023-03-05 15:23:59.106,0 days 01:22:57.257000,9.438625,0 days 00:01:41.016000,278.149997,7,99.0,False,0,5357.235508,...,190.927743,0.35395,0.09953,756,759,0.996047,0.09953,0.35395,783.92720,524.94456
575794,2023-03-05 15:23:59.174,0 days 01:22:57.325000,5.679722,0 days 00:01:41.084000,279.000000,7,99.0,False,0,5362.513611,...,190.927743,0.34333,0.03093,757,759,0.997365,0.03093,0.34333,783.92720,524.94456
575795,2023-03-05 15:23:59.186,0 days 01:22:57.337000,5.679722,0 days 00:01:41.096000,279.119997,7,99.0,False,0,5363.446582,...,190.927743,0.28316,0.00376,758,759,0.998682,0.00376,0.28316,783.92720,524.94456


In [27]:
lap_df.drop(columns=['lon_acc', 'lat_acc', 'abs_lat_acc', 'abs_lon_acc', 'LapProgress'], inplace=True)

In [28]:
lap_df

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,DriverNumber,LapNumber,mean_lap_speed,TimeNumberLapTime,TimeNumberLapCounts,sum_lat_acc,sum_lon_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1,1.0,185.815557,1,762,840.76762,561.07970
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1,1.0,185.815557,2,762,840.76762,561.07970
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1,1.0,185.815557,3,762,840.76762,561.07970
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1,1.0,185.815557,4,762,840.76762,561.07970
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1,1.0,185.815557,5,762,840.76762,561.07970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575792,2023-03-05 15:23:59.014,0 days 01:22:57.165000,14.524167,0 days 00:01:40.924000,277.000000,7,99.0,False,0,5350.113611,-390.939673,1050.884326,-158.999997,81,12.0,190.927743,755,759,783.92720,524.94456
575793,2023-03-05 15:23:59.106,0 days 01:22:57.257000,9.438625,0 days 00:01:41.016000,278.149997,7,99.0,False,0,5357.235508,-388.000000,1111.000000,-159.000000,81,12.0,190.927743,756,759,783.92720,524.94456
575794,2023-03-05 15:23:59.174,0 days 01:22:57.325000,5.679722,0 days 00:01:41.084000,279.000000,7,99.0,False,0,5362.513611,-385.457412,1163.537539,-159.000000,81,12.0,190.927743,757,759,783.92720,524.94456
575795,2023-03-05 15:23:59.186,0 days 01:22:57.337000,5.679722,0 days 00:01:41.096000,279.119997,7,99.0,False,0,5363.446582,-385.000000,1173.000000,-159.000000,81,12.0,190.927743,758,759,783.92720,524.94456


In [ ]:
def get_single_lap_data(df: pd.DataFrame) -> pd.DataFrame:
    final_df = pd.DataFrame(columns=df.columns)

    for driver in df['DriverNumber'].unique():
        driver_df = df[df['DriverNumber'] == driver]
        laps = int(driver_df['LapNumber'].max())

        for lap in range(1, laps + 1):
            lap_df = driver_df[driver_df['LapNumber'] == lap]
            if not lap_df.empty:
                final_row = lap_df.iloc[[-1], :]
                final_df = pd.concat([final_df, final_row], axis=0)

    return final_df

acc_df = get_single_lap_data(lap_df)

acc_df

C:\Users\pat79\AppData\Local\Temp\ipykernel_11616\1187037148.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, final_row], axis=0)


,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,DriverNumber,LapNumber,mean_lap_speed,TimeNumberLapTime,TimeNumberLapCounts,sum_lat_acc,sum_lon_acc
761,2023-03-05 15:05:17.751,0 days 01:04:15.902000,0.000278,0 days 00:01:39.250000,280.142857,7,100.0,False,0,5141.964903,-383.313167,1216.856742,-159.000000,1,1.0,185.815557,762,762,840.76762,561.07970
1499,2023-03-05 15:06:55.725,0 days 01:05:53.876000,NaN,0 days 00:01:37.974000,280.941666,7,100.0,False,0,5363.501061,-383.071911,1222.720301,-158.999999,1,2.0,196.531157,738,738,855.49170,515.23100
2258,2023-03-05 15:10:11.707,0 days 01:09:09.858000,NaN,0 days 00:01:37.976000,280.435716,7,100.0,False,0,5360.387868,-382.866368,1223.632581,-158.999999,1,4.0,197.711395,759,759,853.49005,512.35496
2984,2023-03-05 15:13:27.728,0 days 01:12:25.879000,NaN,0 days 00:01:37.986000,281.299169,7,100.0,False,0,5369.135413,-382.806640,1217.956630,-158.971744,1,6.0,197.696016,726,726,798.17038,506.39443
3714,2023-03-05 15:15:05.749,0 days 01:14:03.900000,NaN,0 days 00:01:38.021000,280.885714,7,100.0,False,0,5369.141600,-382.807112,1225.243520,-159.000006,1,7.0,197.638955,730,730,818.31458,503.46019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572768,2023-03-05 15:17:13.620,0 days 01:16:11.771000,88.106389,0 days 00:01:41.295000,286.574998,7,99.0,False,1,5362.128048,-382.922374,1215.374842,-159.000000,81,8.0,191.880886,785,785,786.42792,581.03253
573511,2023-03-05 15:18:55.154,0 days 01:17:53.305000,73.670278,0 days 00:01:41.534000,286.749997,7,99.0,False,1,5364.900614,-380.141404,1289.632434,-159.000000,81,9.0,190.220934,743,743,771.00349,535.18105
574270,2023-03-05 15:20:36.738,0 days 01:19:34.889000,4.351944,0 days 00:01:41.584000,291.429998,7,99.0,False,1,5348.892812,-382.486468,1230.307245,-159.000000,81,10.0,189.105442,759,759,765.02686,554.66969
575037,2023-03-05 15:22:18.090,0 days 01:21:16.241000,68.856111,0 days 00:01:41.352000,279.468749,7,99.0,False,0,5360.722469,-382.733676,1223.929330,-159.000000,81,11.0,190.452366,767,767,762.02163,521.07765


In [30]:
X_acc = acc_df.loc[:, ['LapNumber', 'sum_lat_acc', 'sum_lon_acc']]

In [ ]:
X_acc.reset_index(inplace=True)
X.reset_index(inplace=True)

In [46]:
X_acc.drop(columns='LapNumber', inplace=True)

In [47]:
X

,index,LapNumber,SpeedI1,SpeedI2,SpeedFL
0,0,1.0,232.0,231.0,278.0
1,1,2.0,227.0,238.0,278.0
2,3,4.0,228.0,238.0,278.0
3,5,6.0,229.0,239.0,279.0
4,6,7.0,230.0,239.0,279.0
...,...,...,...,...,...
753,1050,8.0,225.0,230.0,283.0
754,1051,9.0,223.0,230.0,283.0
755,1052,10.0,225.0,245.0,288.0
756,1053,11.0,222.0,240.0,277.0


In [48]:
X_acc

,level_0,index,sum_lat_acc,sum_lon_acc
0,0,761,840.76762,561.07970
1,1,1499,855.49170,515.23100
2,2,2258,853.49005,512.35496
3,3,2984,798.17038,506.39443
4,4,3714,818.31458,503.46019
...,...,...,...,...
753,753,572768,786.42792,581.03253
754,754,573511,771.00349,535.18105
755,755,574270,765.02686,554.66969
756,756,575037,762.02163,521.07765


In [53]:
X_c = pd.concat([X, X_acc], axis=1)

In [54]:
X_c

,index,LapNumber,SpeedI1,SpeedI2,SpeedFL,level_0,index,sum_lat_acc,sum_lon_acc
0,0,1.0,232.0,231.0,278.0,0,761,840.76762,561.07970
1,1,2.0,227.0,238.0,278.0,1,1499,855.49170,515.23100
2,3,4.0,228.0,238.0,278.0,2,2258,853.49005,512.35496
3,5,6.0,229.0,239.0,279.0,3,2984,798.17038,506.39443
4,6,7.0,230.0,239.0,279.0,4,3714,818.31458,503.46019
...,...,...,...,...,...,...,...,...,...
753,1050,8.0,225.0,230.0,283.0,753,572768,786.42792,581.03253
754,1051,9.0,223.0,230.0,283.0,754,573511,771.00349,535.18105
755,1052,10.0,225.0,245.0,288.0,755,574270,765.02686,554.66969
756,1053,11.0,222.0,240.0,277.0,756,575037,762.02163,521.07765


In [55]:
y

0        99.019
1        97.974
3        97.976
5        97.986
6        98.021
         ...   
1050    101.295
1051    101.534
1052    101.584
1053    101.352
1054    101.156
Name: LapTime_sec, Length: 758, dtype: float64

In [56]:
assert X_c.shape[0] == y.shape[0]

In [59]:
X_scaled = StandardScaler().fit_transform(X)
X_scaled_c = StandardScaler().fit_transform(X_c)

# Split once, using indices
X_train_idx, X_test_idx = train_test_split(range(len(y)), test_size=0.2, random_state=42)

# Apply the split to both versions of X
X_train = X_scaled[X_train_idx]
X_test = X_scaled[X_test_idx]

X_train_c = X_scaled_c[X_train_idx]
X_test_c = X_scaled_c[X_test_idx]

y_train = y.iloc[X_train_idx]
y_test = y.iloc[X_test_idx]

Linear Regression

In [60]:
# Model 1
model1 = LinearRegression()
model1.fit(X_train, y_train)
y_pred_1 = model1.predict(X_test)
mse_1 = mean_squared_error(y_test, y_pred_1)

# Model 2
model2 = LinearRegression()
model2.fit(X_train_c, y_train)
y_pred_2 = model2.predict(X_test_c)
mse_2 = mean_squared_error(y_test, y_pred_2)

print(f"Model 1 MSE (original features): {mse_1:.3f}")
print(f"Model 2 MSE (new features): {mse_2:.3f}")

Model 1 MSE (original features): 34.924
Model 2 MSE (new features): 24.235


In [61]:
print("Model 1 coefficients:", model1.coef_)
print("Model 2 coefficients:", model2.coef_)

Model 1 coefficients: [ 0.31150506 -0.31874986 -2.60315138 -0.91531267 -0.62258379]
Model 2 coefficients: [-8.11029304e+00  1.31353265e-01 -2.97421332e+00 -2.62683059e+00
 -6.15912585e-01  1.47199750e+02 -1.38759563e+02  2.11090436e+00
  2.75028112e+00]


In [62]:
from sklearn.model_selection import cross_val_score

scores1 = cross_val_score(LinearRegression(), X_scaled, y, scoring='neg_mean_squared_error', cv=5)
scores2 = cross_val_score(LinearRegression(), X_scaled_c, y, scoring='neg_mean_squared_error', cv=5)

print(f"Model 1 CV MSE: {-scores1.mean():.3f}")
print(f"Model 2 CV MSE: {-scores2.mean():.3f}")

Model 1 CV MSE: 39.807
Model 2 CV MSE: 30.368


Decision Tree


In [63]:
tree_model = DecisionTreeRegressor(random_state=12)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
print(f"Decision Tree MSE: {mse_tree:.3f}")

tree_model = DecisionTreeRegressor(random_state=12)
tree_model.fit(X_train_c, y_train)
y_pred_tree_c = tree_model.predict(X_test_c)
mse_tree = mean_squared_error(y_test, y_pred_tree_c)
print(f"Decision Tree MSE with additional data: {mse_tree:.3f}")

Decision Tree MSE: 64.829
Decision Tree MSE with additional data: 43.155


Obliczanie submodelu telemetrii w konteście podziału na sektory

In [ ]:
def compute_binned(df : pd.DataFrame, column : str = 'lat', bins : int = 10, aggfunc : str ='mean') -> pd.DataFrame:
    
    df['bin'] = pd.cut(df['LapProgress'], bins=bins, labels=False, include_lowest=True)
    
    result = (
        df.groupby(['LapNumber', 'bin'])[column]
        .agg(aggfunc)
        .reset_index()
        .rename(columns={column: f'{aggfunc}_of_{column}'})
    )
    return result

In [ ]:
binned_result = compute_binned(telemetry, column='lat_acc', bins=10, aggfunc='sum')

binned_result

,LapNumber,bin,sum_of_lat_acc
0,1.0,0,-1.40
1,1.0,1,-27.03
2,1.0,2,-79.70
3,1.0,3,-1.87
4,1.0,4,-30.47
...,...,...,...
565,57.0,5,60.15
566,57.0,6,110.83
567,57.0,7,-165.74
568,57.0,8,-3.58
